In [1]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

model_name = "../../../../projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4"
model_name = os.path.abspath(model_name)
print(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only = True, trust_remote_code = False)
model = AutoModelForCausalLM.from_pretrained(model_name, local_files_only = True, trust_remote_code = False, torch_dtype=torch.bfloat16)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("cuda available:", torch.cuda.is_available())
print("device:", device)
print("done")


/home/sheju347/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.24it/s]


cuda available: True
device: cuda
done


In [2]:
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    context_window=4096,  # Phi-3 Mini typically has a 4k context window
    max_new_tokens=1024,   # Max tokens to generate per response
    generate_kwargs={"temperature": 0.0000001, "do_sample": True},
    # system_prompt="You are a helpful and friendly AI assistant.",
    messages_to_prompt=lambda messages: tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    ),
    # You might need to set `device_map` here as well if not set in AutoModelForCausalLM
    # device_map="cuda" if torch.cuda.is_available() else "cpu",
)

Settings.llm = llm

embed_model = HuggingFaceEmbedding(
    model_name="microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract"
)
Settings.embed_model = embed_model
# embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
# # embedding_model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
# Settings.embed_model = HuggingFaceEmbedding(model_name=embedding_model_name)

print("done")

No sentence-transformers model found with name microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract. Creating a new one with mean pooling.


done


In [3]:

def format_choices(choices):
    a = zip(list(choices.keys()), choices.values())
    final_answers = []
    for x,y in a:
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)


In [1]:
import os

# # Set ALL necessary Hugging Face cache paths
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["HF_HUB_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"
# os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/transformers"
# os.environ["HF_DATASETS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/datasets"


from datasets import load_dataset
ds = load_dataset("MedRAG/pubmed", split = "train")

print ("done")

/home/sheju347/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


done


In [2]:
print(len(ds))
print(ds[0])

23898701
{'id': 'pubmed23n0001_0', 'title': "[Biochemical studies on camomile components/III. In vitro studies about the antipeptic activity of (--)-alpha-bisabolol (author's transl)].", 'content': '(--)-alpha-Bisabolol has a primary antipeptic action depending on dosage, which is not caused by an alteration of the pH-value. The proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1/0.5. The antipeptic action of bisabolol only occurs in case of direct contact. In case of a previous contact with the substrate, the inhibiting effect is lost.', 'contents': "[Biochemical studies on camomile components/III. In vitro studies about the antipeptic activity of (--)-alpha-bisabolol (author's transl)]. (--)-alpha-Bisabolol has a primary antipeptic action depending on dosage, which is not caused by an alteration of the pH-value. The proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1/0.5. The antip

In [ ]:
import sys
from tqdm import tqdm
tqdm._instances.clear()  # clear existing bars
# tqdm.configure(file=sys.stdout, dynamic_ncols=True, leave=False)

from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import time
t1 = time.time()

print(ds)
data_list = ds.select(range(1000000))
docList = []
for data in data_list:
    doc = Document(text = data["contents"], 
                   metadata={"title":data["title"]}, 
                   id_=data["id"])
    docList.append(doc)

index = VectorStoreIndex.from_documents(documents = docList, embed_model = embed_model, show_progress = True)
index.storage_context.persist("/projects/sciences/computing/sheju347/RAG/pubmed_index_1000k")

t2 = time.time()
print(t2 - t1)


Dataset({
    features: ['id', 'title', 'content', 'contents', 'PMID'],
    num_rows: 23898701
})


Generating embeddings:  40%|███▉      | 817/2048 [00:04<00:09, 128.47it/s]

In [6]:

# example = {"question": "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complications and has not had any medical problems up until this time. However, for the past 4 days, he has been fussy, is regurgitating all of his feeds, and his vomit is yellow in color. On physical exam, the child's abdomen is minimally distended but no other abnormalities are appreciated. Which of the following embryologic errors could account for this presentation?",
#         "choices": {
#         "A": "Abnormal migration of ventral pancreatic bud",
#         "B": "Complete failure of proximal duodenum to recanalize",
#         "C": "Abnormal hypertrophy of the pylorus",
#         "D": "Failure of lateral body folds to move ventrally and fuse in the midline"
#         }
# }


# example = {"question": """Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise. He has type 2 diabetes mellitus and osteoarthritis of the hips. Prior to admission, his medications were insulin and naproxen. He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention. His temperature is 38°C (100.4°F), pulse is 93/min, and blood pressure is 125/85 mm Hg. Examination shows mottled, reticulated purplish discoloration of the feet. Laboratory studies show:
# Hemoglobin count 14 g/dL
# Leukocyte count 16,400/mm3
# Segmented neutrophils 56%
# Eosinophils 11%
# Lymphocytes 31%
# Monocytes 2%
# Platelet count 260,000/mm3
# Erythrocyte sedimentation rate 68 mm/h
# Serum
# Urea nitrogen 25 mg/dL
# Creatinine 4.2 mg/dL
# Renal biopsy shows intravascular spindle-shaped vacuoles. Which of the following is the most likely cause of this patient's symptoms?""",
#            "choices": {
#             "A": "Renal papillary necrosis",
#             "B": "Cholesterol embolization",
#             "C": "Eosinophilic granulomatosis with polyangiitis",
#             "D": "Polyarteritis nodosa"
#             }
#         }


example = {"question": "A 46-year-old man is brought to the emergency department for evaluation of altered mental status. He was found on the floor in front of his apartment. He is somnolent but responsive when aroused. His pulse is 64/min, respiratory rate is 15/min, and blood pressure is 120/75 mm Hg. On physical examination, an alcoholic smell and slurred speech are noted. Neurological exam shows diminished deep tendon reflexes bilaterally and an ataxic gait. His pupils are normal. Blood alcohol concentration is 0.04%. An ECG shows no abnormalities. Which of the following is the most likely cause of this patient's symptoms?",
           "choices": {
               "A": "Epstein-Barr virus",
            "B": "Human immunodeficiency virus",
            "C": "Rotavirus",
            "D": "Vaccinia virus"
        }
}


prompt_eval_context_bare = f'''
{{context}} \n
{{question}} \n
{{choices}}
'''
            
formated_choices = format_choices(example["choices"])

# question = "cause of type 2 diabetes"
# context = "The cause of type 2 diabetes is lack of activity and excessive intake of sugar"
question = example["question"]
context = ""
contexts = [
    # "Complete failure of proximal duodenum to recanalize would be presented much earlier",
    # "Hypertrophic pyloric stenosis) causes non-bilious projectile vomiting due to gastric outlet obstruction",
           ]
for c in contexts:
    context += c + "\n"

choices = example["choices"]
prompt = prompt_eval_context_bare.format(question = question, context = context, choices = formated_choices)

print(prompt)


 

A 46-year-old man is brought to the emergency department for evaluation of altered mental status. He was found on the floor in front of his apartment. He is somnolent but responsive when aroused. His pulse is 64/min, respiratory rate is 15/min, and blood pressure is 120/75 mm Hg. On physical examination, an alcoholic smell and slurred speech are noted. Neurological exam shows diminished deep tendon reflexes bilaterally and an ataxic gait. His pupils are normal. Blood alcohol concentration is 0.04%. An ECG shows no abnormalities. Which of the following is the most likely cause of this patient's symptoms? 

[A] : Epstein-Barr virus
[B] : Human immunodeficiency virus
[C] : Rotavirus
[D] : Vaccinia virus



In [4]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir = "/projects/sciences/computing/sheju347/RAG/pubmed_index_100k")
index = load_index_from_storage(storage_context)

print(index)


In [7]:

query_engine = index.as_query_engine()
response = query_engine.query(prompt)
print(response.response)
print(type(response))


# Print the retrieved source nodes
for i, node in enumerate(response.source_nodes):
    print(f"--- Source {i + 1} ---")
    print(node.text)
    print(f"Score: {node.score}")  # Optional: similarity score
    print()



The patient presents with altered mental status, somnolence, slurred speech, diminished deep tendon reflexes, and an ataxic gait. The blood alcohol concentration is low at 0.04%, which makes alcohol intoxication an unlikely cause of his symptoms. The ECG shows no abnormalities, which makes cardiac causes less likely.

The patient's symptoms do not align well with Epstein-Barr virus (A), which typically causes infectious mononucleosis, or with Rotavirus (C), which is a common cause of gastroenteritis in children and not associated with altered mental status or neurological symptoms in adults. Vaccinia virus (D) is associated with smallpox and its vaccine, neither of which is relevant to the patient's presentation.

Human immunodeficiency virus (B), on the other hand, can cause a wide range of neurological symptoms, including altered mental status, ataxia, and diminished deep tendon reflexes. These symptoms can be due to the direct effects of the virus on the nervous system or secondar